In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from tqdm.notebook import tqdm
tqdm.pandas()
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning)
import nltk
from textblob import TextBlob
import matplotlib.pyplot as plt
import string

# Import Data

In [2]:
data = pd.read_csv("/kaggle/input/consumer-complaint/complaints.csv")
data = data[data["Consumer complaint narrative"].isnull()==False]
data.reset_index(inplace=True, drop=True)
# data["Consumer complaint narrative"][np.random.randint(len(data),size=1)].values
print("Count of unique Issues: ", len(data["Issue"].unique()))
# data = data[0:3000]

Count of unique Issues:  160


# Data Cleaning

In [ ]:
data["Consumer complaint narrative"]

# Lowercase
data["Consumer complaint narrative"] = data["Consumer complaint narrative"].progress_apply(lambda x: " ".join(x.lower() for x in x.split()))

# Removing Punctuation
data["Consumer complaint narrative"] = data["Consumer complaint narrative"].progress_apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

# # Removal of word having two or more "x" letters
data["Consumer complaint narrative"] = data["Consumer complaint narrative"].progress_apply(lambda x: " ".join(x for x in x.split() if "x" not in x))

# Removal of numbers and words having numbers
data["Consumer complaint narrative"] = data["Consumer complaint narrative"].progress_apply(lambda x: " ".join(x for x in x.split() if not any(c.isdigit() for c in x)))

data["Consumer complaint narrative"][100:1100].values

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

  0%|          | 0/3000 [00:00<?, ?it/s]

# Data Visualization

In [ ]:
# # Pie chart
# labels = data["Timely response?"].unique()
# sizes = [len(data[data["Timely response?"]==labels[0]]), len(data[data["Timely response?"]==labels[1]])]
# explode = (0.1, 0)  # explode 1st slice
# # Plot
# plt.pie(sizes, explode=explode, labels=labels, autopct='%1.1f%%', shadow=False, startangle=140)
# plt.axis('equal')
# plt.title("Timely Response")
# plt.show()

In [ ]:
# # Histogram for Categorical Variable
# data["Company response to consumer"].value_counts().plot(kind='bar', title='Company response to consumer', figsize=(14, 6), rot=0, width=0.3 )

In [ ]:
# data[['Company response to consumer',"Complaint ID"]].groupby(by="Company response to consumer").count().sort_values("Complaint ID").plot(kind='bar', figsize=(15, 5), rot=0)
# data[['State',"Complaint ID"]].groupby(by="State").count().sort_values("Complaint ID").plot(kind='bar', figsize=(25, 5), rot=0)


# # Change into datatime format
# data["Date received"] = pd.to_datetime(data["Date received"])
# data_temp = data[["Complaint ID","Date received"]]
# data_temp.sort_values(by=["Date received"],inplace=True)
# # Create year week column
# data_temp["year_week"] = data_temp["Date received"].dt.strftime('%Y-%U')
# data_temp = data_temp[["Complaint ID","year_week"]].groupby(by="year_week").count().rename(columns={"Complaint ID":"count"})
# data_temp.plot(figsize=(15,5), rot=0, title="Complaints per week")

In [ ]:
# data_temp = data["Consumer complaint narrative"].apply(lambda x: TextBlob(x).sentiment.polarity)
# plt.hist(data_temp, bins=50)
# plt.show()

In [ ]:
# # Uni-gram using CountVectorizer
# from sklearn.feature_extraction.text import CountVectorizer
# cv = CountVectorizer(ngram_range=(1,1), stop_words='english')
# cv.fit(data["Consumer complaint narrative"])
# sum_words = cv.transform(data["Consumer complaint narrative"]).sum(axis=0)
# words_freq = [(word, sum_words[0, idx]) for word, idx in cv.vocabulary_.items()]
# words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
# # plot histogram for top 20 words
# plt.figure(figsize=(15,5))
# plt.bar([i[0] for i in words_freq[:20]], [i[1] for i in words_freq[:20]])
# plt.xticks(rotation=30)
# plt.show()

In [ ]:
# # Bi-gram
# cv = CountVectorizer(ngram_range=(2,2), stop_words='english')
# cv.fit(data["Consumer complaint narrative"])
# sum_words = cv.transform(data["Consumer complaint narrative"]).sum(axis=0)
# words_freq = [(word, sum_words[0, idx]) for word, idx in cv.vocabulary_.items()]
# words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
# # plot histogram for top 20 words
# plt.figure(figsize=(15,5))
# plt.bar([i[0] for i in words_freq[:20]], [i[1] for i in words_freq[:20]])
# plt.xticks(rotation=40)
# plt.show()

In [ ]:
# from wordcloud import WordCloud, STOPWORDS
# stopwords = set(STOPWORDS)

# wordcloud = WordCloud(width = 800, height = 800, stopwords=stopwords, max_words=100, max_font_size=40, random_state=10)

# wordcloud=wordcloud.generate(" ".join( data['Consumer complaint narrative'].values) )

# fig = plt.figure(1, figsize=(12, 12))
# plt.axis('off')
# plt.imshow(wordcloud)
# plt.show()

In [ ]:
data = data[["Issue", "Consumer complaint narrative"]]
data = data[(data["Consumer complaint narrative"]!="")] 
data.reset_index(inplace=True, drop=True)

In [ ]:
# Save cleaned data
data.to_csv("complaints_cleaned.csv", index=False)

# Data Preprocessing

In [ ]:
import numpy as np
import pandas as pd 
import re
import gensim
from gensim import corpora, models, similarities
from gensim.parsing.preprocessing import STOPWORDS
from nltk.corpus import stopwords

In [ ]:
# Load cleaned data
data = pd.read_csv("/kaggle/working/complaints_cleaned.csv")
# data.rename(columns={"Consumer complaint narrative":'text'},inplace=True)
column = "Consumer complaint narrative"
data

In [ ]:
# Removing stopwords

import nltk
# nltk.download('stopwords')
import gensim
from gensim.parsing.preprocessing import STOPWORDS
from nltk.corpus import stopwords
 
stop_words = stopwords.words('english')

# Remove stop words
data[column] = data[column].progress_apply(lambda x: " ".join(x for x in x.split() if x not in stop_words))

In [ ]:
# # Build the bigram and trigram models
# bigram = gensim.models.Phrases(data[column].values, min_count=5, threshold=100) # higher threshold fewer phrases.
# trigram = gensim.models.Phrases(bigram[column], threshold=100)  

# # Faster way to get a sentence clubbed as a trigram/bigram
# bigram_mod = gensim.models.phrases.Phraser(bigram)
# trigram_mod = gensim.models.phrases.Phraser(trigram)

# def make_bigrams(texts):
#     return [bigram_mod[doc] for doc in texts]

# def make_trigrams(texts):
#     return [trigram_mod[bigram_mod[doc]] for doc in texts]

# def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     texts_out = []
#     for sent in texts:
#         doc = nlp(" ".join(sent)) 
#         texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#     return texts_out

In [ ]:
# Lemmatization using spacy
import spacy
nlp = spacy.load('en_core_web_sm',disable=['ner', 'parser']) # conda install -c conda-forge spacy-model-en_core_web_lg
def lemmatize_words(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])
data["text_lemmatized"] = data[column].progress_apply(lambda text: lemmatize_words(text))

# Tokinization using spacy
def tokenize(text):
    doc = nlp(text)
    return [token.text for token in doc]
data["text_lemmatized"] = data["text_lemmatized"].progress_apply(lambda text: tokenize(text))

In [19]:
data["text_lemmatized"]

0       [minimum, payment, fortiva, retail, credit, ca...
1       [identity, steal, someone, create, fraudelunet...
2       [I, m, send, complaint, inform, credit, bureau...
3       [identity, theft, florida, date, birth, ss, fl...
4       [file, numerous, complaint, credit, bureau, st...
                              ...                        
2994    [value, set, aside, margin, remove, portion, d...
2995    [name, complaint, make, error, neither, make, ...
2996    [accordance, fair, credit, reporting, act, lis...
2997    [I, m, honestly, stump, regardless, many, time...
2998    [fcra, violation, name, address, mine, fcra, v...
Name: text_lemmatized, Length: 2999, dtype: object

# Creating Document Term Matrix | Gensim

In [20]:
import gensim
from gensim import corpora
 
# Create Dictionary
id2word_dictionary = corpora.Dictionary(data['text_lemmatized'].values)

# Term Document Frequency
corpus = [id2word_dictionary.doc2bow(text) for text in data['text_lemmatized'].values]

# View
print(corpus[0])

[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 2), (12, 1), (13, 5), (14, 1), (15, 2), (16, 1), (17, 8), (18, 4), (19, 5), (20, 1), (21, 1), (22, 1), (23, 3), (24, 8), (25, 1), (26, 2), (27, 3), (28, 2), (29, 2), (30, 1), (31, 2), (32, 2), (33, 1), (34, 2), (35, 1)]


# Implementation of LDA - Latent Dirichlet Allocation

In [21]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=id2word_dictionary, num_topics=10, random_state=10, 
                                        chunksize=100, passes=10, per_word_topics=True)

In [22]:
from pprint import pprint

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.039*"identity" + 0.037*"theft" + 0.037*"balance" + 0.036*"information" + '
  '0.035*"report" + 0.022*"consumer" + 0.022*"item" + 0.020*"block" + '
  '0.020*"file" + 0.018*"require"'),
 (1,
  '0.070*"fargo" + 0.070*"wells" + 0.039*"charge" + 0.023*"amount" + 0.016*"I" '
  '+ 0.013*"transaction" + 0.012*"request" + 0.012*"product" + 0.010*"finance" '
  '+ 0.010*"agreement"'),
 (2,
  '0.034*"account" + 0.023*"bank" + 0.021*"call" + 0.014*"tell" + '
  '0.013*"receive" + 0.013*"check" + 0.012*"card" + 0.012*"would" + '
  '0.010*"say" + 0.009*"not"'),
 (3,
  '0.042*"credit" + 0.017*"debt" + 0.014*"company" + 0.010*"collection" + '
  '0.010*"report" + 0.010*"letter" + 0.010*"send" + 0.009*"one" + 0.008*"pay" '
  '+ 0.008*"card"'),
 (4,
  '0.092*"payment" + 0.061*"loan" + 0.031*"pay" + 0.028*"mortgage" + '
  '0.027*"late" + 0.023*"month" + 0.022*"not" + 0.017*"make" + 0.016*"car" + '
  '0.015*"do"'),
 (5,
  '0.029*"report" + 0.024*"use" + 0.023*"consent" + 0.021*"make" + '
  '0.018*"

# Evaluation | C_v Score

In [23]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data["text_lemmatized"].values, dictionary=id2word_dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.5502356648213762


# Hyperparameter Tuning

In [ ]:
#  Function for TRaining LDA model and Computing Coherence Score

def Train_LDA_Compute_Coherence(corpus, dictionary, data_lemmatized, num_of_topics, alpha, beta, num_of_paases, coherence='c_v'):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus, id2word=dictionary, num_topics=num_of_topics, random_state=10,
                                           chunksize=100, passes=num_of_paases, alpha=alpha, eta=beta)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=dictionary, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

score = Train_LDA_Compute_Coherence(corpus=corpus, dictionary=id2word_dictionary, data_lemmatized=data["text_lemmatized"].values, 
                                        num_of_topics=20, alpha=1, beta=1, num_of_paases=15)
score

In [ ]:
# Create Grid of Hyperparameters

# Topics range
min_topics = 2
max_topics = 5
step_size = 1
topics_range = [5,25,50,100,150] # range(min_topics, max_topics, step_size)

# Alpha parameter
alpha_options = list(np.arange(0.01, 1, 0.475))
alpha_options.append('symmetric')
alpha_options.append('asymmetric')

# Beta parameter
beta_options = list(np.arange(0.01, 1, 0.475))
beta_options.append('symmetric')

tuning_results = pd.DataFrame(columns=['Number of Topics', 'Alpha', 'Beta'])

# Create Array of possible combination of above parameters
hyperparameters = np.array( np.meshgrid(alpha_options, beta_options, topics_range)).T.reshape(-1, 3)
tuning_results['Alpha']= hyperparameters[:,0]
tuning_results['Beta']= hyperparameters[:,1]
tuning_results['Number of Topics']= hyperparameters[:,2]
tuning_results

In [ ]:
# Tuning Process

stages_of_passes = [2,5,10]
num_of_tops_hyperparameter = [5,3]
num_of_tops_hyperparameter.append( num_of_tops_hyperparameter[-1] )
tuning_df = tuning_results.copy()

for stage in range(0,len(stages_of_passes)):
    print(  )
    scores = []
    for i in tqdm(range(0,tuning_results.shape[0])):
        hyperparameter = tuning_results.iloc[i]

        try:
            Alpha = float(hyperparameter['Alpha'])
        except:
            Alpha = hyperparameter['Alpha']

        try:
            Beta = float(hyperparameter['Beta'])
        except:
            Beta = hyperparameter['Beta']
        
        try:
            Number_of_Topics = int(hyperparameter['Number of Topics'])
        except:
            Number_of_Topics = hyperparameter['Number of Topics']

        
        score = Train_LDA_Compute_Coherence(corpus=corpus, dictionary=id2word_dictionary, data_lemmatized=data["text_lemmatized"].values, 
                                            num_of_topics=Number_of_Topics, alpha=Alpha, beta=Beta, num_of_paases=stages_of_passes[stage])
        scores.append(score)

    tuning_df["coherence score stage " +  str(stage)] = score
    tuning_df.sort_values(by="coherence score stage " + str(stage),ascending=False)

    tuning_df = tuning_df.head(num_of_tops_hyperparameter[stage])

tuning_df

In [ ]:
# Best Model
best_hyperparameters = tuning_df.iloc[0].to_dict()
print("Best Hyperparameters: \n", best_hyperparameters)

try:
    best_hyperparameters['Alpha'] = float(best_hyperparameters['Alpha'])
except:
    pass

try:
    best_hyperparameters['Beta'] = float(best_hyperparameters['Beta'])
except:
    pass

best_hyperparameters['Number of Topics'] = int( best_hyperparameters['Number of Topics'] )

# Train Best Model
lda_best_model = gensim.models.LdaMulticore(corpus=corpus, id2word=id2word_dictionary, num_topics=int(best_hyperparameters['Number of Topics']), random_state=10,
                                             chunksize=100, passes=stages_of_passes[-1], alpha=best_hyperparameters['Alpha'], eta=best_hyperparameters['Beta'])

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_best_model, texts=data["text_lemmatized"].values, dictionary=id2word_dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\n \n Coherence Score of Best Model:-    c_v score= ', coherence_lda, "  |   u_mass score= ", lda_best_model.log_perplexity(corpus))

# Save Best Model
lda_best_model.save("lda_best_model")

In [ ]:
# List topics modelled by Best Model
pprint(lda_best_model.print_topics())

In [ ]:
# Print Document and corresponding Topic
for i in range(0,10):
    print("Document: ", data[column][i])
    print("Topic: ", lda_best_model[corpus[i]])
    print(" ")

# Visualization

In [ ]:
# Visualize the topics
import pyLDAvis
pyLDAvis.enable_notebook()

vis_graph = pyLDAvis.gensim_models.prepare(lda_best_model, corpus, id2word_dictionary)
vis_graph

# LSA - Latent Semantic Analysis

In [ ]:
from  import LsiModel

lsi_model = LsiModel(corpus=corpus, num_topics=200, id2word=id2word_dictionary, chunksize=100, distributed=False, random_seed=10, onepass=True, power_iters=2)
coherence_model_lsi = CoherenceModel(model=lsi_model, texts=data['text_lemmatized'].values, dictionary=id2word_dictionary, coherence='c_v')
coherence_model_lsi.get_coherence()

In [ ]:
# Function for Training LDA model and Computing Coherence Score

def Train_LSI_Compute_Coherence(corpus, dictionary, data_lemmatized, num_of_topics, coherence='c_v'):
    
    lsi_model = gensim.models.lsimodel.LsiModel(corpus=corpus, id2word=dictionary, num_topics=num_of_topics,
                                                chunksize=100, distributed=False, random_seed=10, onepass=True, power_iters=2)
    
    coherence_model_lsi = CoherenceModel(model=lsi_model, texts=data_lemmatized, dictionary=dictionary, coherence='c_v')
    
    return coherence_model_lsi.get_coherence()

score = Train_LSI_Compute_Coherence(corpus=corpus, dictionary=id2word_dictionary, data_lemmatized=data["text_lemmatized"].values, num_of_topics=20)
score

In [ ]:
# Finding Optimal Number of Topics

# Topics range
min_topics = 2
max_topics = 5
step_size = 1
topics_range = [5,25,50,100,150]# range(min_topics, max_topics, step_size)

scores = []
for i in tqdm(range(0,len(topics_range))):
    score = Train_LSI_Compute_Coherence(corpus=corpus, dictionary=id2word_dictionary, data_lemmatized=data["text_lemmatized"].values, num_of_topics=topics_range[i])
    scores.append(score)

# Plotting
plt.figure(figsize=(10,5))
plt.plot(topics_range)
plt.xlabel("Number of Topics")
plt.ylabel("Coherence Score")
plt.title("Coherence Scores for LSI Model")
plt.show()


In [ ]:
# Final LSI Model
optimal_number_of_topics = topics_range[np.argmax(scores)]
best_lsi_model = gensim.models.lsimodel.LsiModel(corpus=corpus, id2word=id2word_dictionary, num_topics=optimal_number_of_topics, chunksize=100, distributed=False, random_seed=10, onepass=True, power_iters=2)
print( " Optimal Number of Topics: ", optimal_number_of_topics )
# Compute Coherence Score
coherence_model_lsi = CoherenceModel(model=best_lsi_model, texts=data["text_lemmatized"].values, dictionary=id2word_dictionary, coherence='c_v')
coherence_lsi = coherence_model_lsi.get_coherence()
print('\n Coherence Score of Best Model:-    c_v score= ', coherence_lsi )

# save best model
best_lsi_model.save("best_lsi_model")                   

In [ ]:
# List topics modelled by Best Model
pprint(best_lsi_model.print_topics())

***